# Pregunta 3 - Tarea 1
## Estudiante: Diego Emilio Bustamante Henríquez

### Merkle Tree

Se decide almacenar los parámetros del árbol y calcular los output de las funciones al mismo tiempo que se contruye el estructura.

`get_root`: calcula el hash de las hojas a partir de los strings. Siempre que no se haya alcanzado la root, se ajusta la paridad de la lista y se calcula el siguiente nivel del árbol emparejando de a 2 y calculando los hash (el zip permite emparejar los nodos pares con su correspontiente impar). Al llegar a un nivel con un nodo, se retorna la root.

`get_proof_for`: método parecido al anterior. Se revisa que el item este entre los strings y se obtiene su posición en la lista. A medida que se calculan los niveles del árbol, se busca el nodo vecino (según la paridad del índice) y se añade a la proof. Tanto en los nodos pares como impares el índice del padre es la división entera del índice del nodo. Al llegar al nivel root, se retorna la proof. 

In [62]:
class MerkleTree :
  def __init__(self, strings, hash_func):
    self.strings = strings
    self.hash_func = hash_func

  def get_root(self):
    nodes = [self.hash_func(elemento) for elemento in self.strings]
    while len(nodes) > 1:
       if len(nodes)%2 != 0: nodes.append(nodes[-1])
       nodes = [self.hash_func(elem1 + elem2) for elem1,elem2 in zip(nodes[0::2], nodes[1::2])]
    return nodes[0]

  def get_proof_for(self, item):
    if item not in self.strings: return None
    index = self.strings.index(item)
    proof = []
    nodes = [self.hash_func(elemento) for elemento in self.strings]
    while len(nodes) > 1:
       if len(nodes)%2 != 0: nodes.append(nodes[-1])
       if index%2 == 0: proof.append((nodes[index + 1], "d"))
       else: proof.append((nodes[index - 1], "i"))
       index >>= 1
       nodes = [self.hash_func(elem1 + elem2) for elem1,elem2 in zip(nodes[0::2], nodes[1::2])]
    return proof

### Verify

Si existe una proof, se calcula el hash del item. Luego, se itera sobre la proof para calcular el hash acumulado dependiendo de si es un nodo a la izquierda o derecha. Si este hash resulta ser el root se retorna True, se retorna False en caso contrario.

In [76]:
def verify (root, item, proof, hash_func):
  if not proof: return False
  cummulative_hash = hash_func(item)
  for value, dir in proof:
    if dir == "d": cummulative_hash = hash_func(cummulative_hash + value)
    else: cummulative_hash = hash_func(value + cummulative_hash)
  if cummulative_hash == root: return True
  return False

### Test

Funciones y cálculos para testear el código.

In [81]:
"""
from hashlib import sha256
def test_fn(a):
  return sha256(a.encode('utf-8')).hexdigest()
# verify(test_fn(test_fn(test_fn("gato") + test_fn("piolin")) + test_fn("perro")), "piolin", [(test_fn("gato"), "i"), (test_fn("perro"), "d")], test_fn)
tree = MerkleTree(["gato", "piolin", "perro", "pez"], test_fn)
print(tree.get_root())
print(tree.get_proof_for("perro"))
print(verify(tree.get_root(), "pez", tree.get_proof_for("pez"), test_fn))
"""

f438cd3b70cd1c623bfac0578bbad20bde969d1d76bcdfb2c2ecb6e338398220
[('472353467f2c5636fd2dfe9f6cbb2520d0e81654e2a0064cf48f277eb0b97e17', 'd'), ('38b7911e451e23b27322f0b03f22de8394cff89d859e406e4d9d7439b45db276', 'i')]
True
